In [ ]:
# import torch

# from learning.network.sac import *

# policy = Policy()
# qf = QFunction()

# image_feature = torch.randn(5, 512)
# object_feature = torch.randn(5, 512)
# action = torch.randn(5, 2)

# policy(image_feature, object_feature)

# qf(image_feature, object_feature, action)

In [ ]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__

In [ ]:
import getpass
user = getpass.getuser()
print(user)

In [ ]:
usd_path = f"omniverse://localhost/Users/{user}/uva_sac.usd"

In [ ]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

In [ ]:
from uva_env import UvaEnv
env = UvaEnv()

In [ ]:
from task.utils import add_scene_default
add_scene_default()

In [ ]:
print(list(env.stage.TraverseAll()))

In [ ]:
env.clean()
env.world.step(render=True)

# Config

In [ ]:
from task.config import DATA_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

# Scene

In [ ]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [ ]:
# add base
scene.add_base_asset()

In [ ]:
# add room
# scene.add_room()

In [ ]:
env.world.step(render=True)

# Reward

In [ ]:
from uv.reward import Rewarder

In [ ]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [ ]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras()
render.set_cameras()

In [ ]:
pause

# Trajectory

In [ ]:
# traj config
traj_id = 0

base_asset_id = 0
env.scene.base_asset_id = base_asset_id
env.scene.traj_id = traj_id
image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

# base
# scene.add_base_asset()
env.world.step(render = True)

# get images
env.world.render()
images = render.get_images()
render.save_rgb(images[0]['rgb'], image_folder, "0")

In [ ]:
# trajectory
for step in range(5):

    env.add_scene_obj(mode = "random")

    x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())

    env.put_last_object((x, y)) 
    env.world.step(render=True)

    env.register_last_object()
    env.world.step(render=True)
    
    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

    # calculate reward
    env.calculate_last_reward()

    env.world.reset()
    env.world.step(render=True)


    

# Record
record = env.scene.get_scene_data()
env.scene.save_scene_data()
print("record: ", record)

# Reset (env clean)
env.clean(clean_all = False)
env.step(render = True)

# Debug

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))